# Excercise 08: Release Strategies


### Create Gateway Http Route to Eligibility Service

Create a new file in the folder `pipelines` named `eligibility-service-route.yml`:

```yaml
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: eligibility-service-route
spec:
  parentRefs:
    - name: mlprod-gateway
      namespace: envoy-gateway-system
  rules:
    - matches:
        - path:
            type: PathPrefix
            value: /<yourname>
      filters:
        - type: URLRewrite
          urlRewrite:
            path:
              type: ReplacePrefixMatch
              replacePrefixMatch: /
      backendRefs:
        - name: eligibility-service
          port: 9000
```

Replace `<yourname>` with your name.

This will create a new route that is exposed over the Api Gateway. Check the deployment state of your model:

`kubectl get httproute`

If you see your route is configured you can continue testing.

### Test deployment

Our dummy deployment can now be accessed using the api gateway. Execute the following line after replacing `<yourname>` with your name.

You should see something like `{"data":{"names":[],"ndarray":[1]},"meta":{"requestPath":{"classifier":"...."}}}`
which is the result of your model.



In [ ]:
!wget -O- mlproduction.dsiag.ch/<yourname>/api/v1.0/predict --post-data '{"data": { "ndarray": [[25000, 189625]]}}' --header='Content-Type:application/json'

## Reconfigure Kubernetes deployment

If you want that your Sustaino frontend uses the newly deployed service, you need to change `pipeline/frontend.yml`

Change

```yaml
env:
    - name: ELIGIBILITY_API_ENDPOINT
      value: http://eligibility-service:8100/predict
```

to:

```yaml
env:
    - name: ELIGIBILITY_API_ENDPOINT
      value: http://mlproduction.dsiag.ch/<yourname>/predict
```


Remember to replace `<yourname>`

Apply the configuration using `kubectl apply -f pipeline/frontend.yml`

## Deploy two versions of the same service

Create a new file in the folder `pipeline` named `ab-eligibility-service-model.yml`:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: eligibility-service-deployment
  labels:
    app: eligibility-service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: eligibility-service
  template:
    metadata:
      labels:
        app: eligibility-service
        monitor: "true"
    spec:
      containers:
        - name: eligibility
          image:  gcr.io/academy-300812/<yourname>/eligibility-service:latest
          ports:
            - name: web
              containerPort: 9000
---
kind: Service
apiVersion: v1
metadata:
  name: eligibility-service
spec:
  selector:
    app: eligibility-service
  ports:
    - protocol: TCP
      port: 9000
      targetPort: 9000
  type: ClusterIP
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: eligibility-service-deployment
  labels:
    app: dummy-eligibility-service
spec:
  replicas: 1
  selector:
    matchLabels:
      app: dummy-eligibility-service
  template:
    metadata:
      labels:
        app: dummy-eligibility-service
        monitor: "true"
    spec:
      containers:
        - name: eligibility
          image: fluescher/sustaino-dummy-service
          ports:
            - name: web
              containerPort: 9000
---
kind: Service
apiVersion: v1
metadata:
  name: dummy-eligibility-service
spec:
  selector:
    app: dummy-eligibility-service
  ports:
    - protocol: TCP
      port: 9000
      targetPort: 9000
  type: ClusterIP

```

Apply this configuration: `kubectl apply -f pipeline/ab-eligibility-service-model.yml`

Verify that the pods are started correctly by issueing `kubectl get pods`. Are all up and running?


## Configure Gateway to serve both models


Create a new file in the folder `pipelines` named `ab-eligibility-service-route.yml`:

```yaml
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: eligibility-service-route
spec:
  parentRefs:
    - name: mlprod-gateway
      namespace: envoy-gateway-system
  rules:
    - matches:
        - path:
            type: PathPrefix
            value: /<yourname>
      filters:
        - type: URLRewrite
          urlRewrite:
            path:
              type: ReplacePrefixMatch
              replacePrefixMatch: /
      backendRefs:
        # Destination 1 (50%)
        - name: eligibility-service
          port: 9000
          weight: 50
        # Traffic Destination 2 (50%)
        - name: dummy-eligibility-service
          port: 9000
          weight: 50
```

You can generate load: `NAMESPACE=<yourname> ../services/create-load.sh`


Replace `<yourname>` with your name.

Questions:
- Generate some load to this model. Can you verify if both deployments get traffic? Check Prometheus
- How is the user impacted by this deployment?



## Shadowing

Create a new file in the folder `pipelines` named `shadow-eligibility-service-route.yml`:


```yaml
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: eligibility-service-route
spec:
  parentRefs:
    - name: mlprod-gateway
      namespace: envoy-gateway-system
  rules:
    - matches:
        - path:
            type: PathPrefix
            value: /<yourname>
      filters:
        - type: URLRewrite
          urlRewrite:
            path:
              type: ReplacePrefixMatch
              replacePrefixMatch: /
        - type: RequestMirror
          requestMirror:
            backendRef:
              kind: Service
              name: eligibility-service
              port: 9000
      backendRefs:
        - name: dummy-eligibility-service
          port: 9000
```

Apply this configuration: `kubectl apply -f pipeline/shadow-eligibility-service-route.yml`

Verify that the pods are started correctly by issueing `kubectl get pods`. Are all up and running?

You can generate load: `NAMESPACE=<yourname> ../services/create-load.sh`


Questions:
- Generate some load to this model. Can you verify if both deployments get traffic? Check Prometheus
- How is the user impacted by this deployment?